In [1]:
# Associate words with archetypes/character traits as intermediate layer
# and with influencer as the "last" layer

# Dependencies
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import os
import toml
import re
import itertools
from text_cleaner import *
import operator
from collections import Counter
from spmf import Spmf

def extract_hashtags(post_text):
    HASH_RE = re.compile(r"\#\w+")
    out_list = re.findall(HASH_RE, post_text)
    return out_list

In [2]:
# Load the .csv with archetypes
arch_df = pd.read_csv('archetypes_pl.csv', index_col=0)

# Save the order of columns
trait_list = arch_df.columns.tolist()

# Show the table header and column list
print(trait_list)
arch_df.head()

['innocent', 'sage', 'explorer', 'outlaw', 'magician', 'hero', 'lover', 'jester', 'everyman', 'caregiver', 'ruler', 'creator', 'dominant', 'submissive', 'maximalist', 'minimalist', 'inspiring', 'systematic', 'discovering', 'conservative', 'verifying', 'overlooking', 'sharpening', 'harmonic', 'empathic', 'matter_of_fact', 'brave', 'protective', 'generous', 'thrifty', 'favourable', 'balanced', 'sensuality', 'intelligent', 'believe', 'egocentric', 'allocentric']


,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [3]:
# Table preprocessing - replace all NaN with 2 (Unrelated/Don't know class), replace 0-5 values with the ones in range -1.0 - 1.0
arch_df = arch_df.fillna(2.0)

arch_df = arch_df.replace(0.0, -1.0)
arch_df = arch_df.replace(1.0, -0.5)
arch_df = arch_df.replace(2.0, 0.0)
arch_df = arch_df.replace(3.0, 0.5)
arch_df = arch_df.replace(4.0, 1.0)

# Remove duplicated annotations, to exclude conflicting entries
arch_df = arch_df[~arch_df.index.duplicated(keep='first')]

# Print the head of the dataset after modification
arch_df.head()

,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,...,0.0,0.5,1.0,1.0,0.5,1.0,1.0,0.5,-1.0,-1.0
vege_style_life,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,1.0,1.0,1.0,0.5,0.5,0.5,0.0,-1.0,0.5
oliwka__2007,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,...,0.0,0.0,-1.0,0.5,-0.5,0.0,1.0,-0.5,-1.0,0.5
z_przestrzeni_serca,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.0,0.5,-1.0,1.0,1.0,0.5,1.0,1.0,-1.0,-0.5
zaradne_warsztaty,0.5,-1.0,-1.0,-1.0,0.5,-1.0,-1.0,0.0,0.5,1.0,...,0.5,1.0,-1.0,0.0,0.0,1.0,0.0,0.5,-0.5,0.5


In [4]:
# Check if a user has a non-empty directory in the dataset, otherwise delete the user from the list
available_arch_df = copy.deepcopy(arch_df)
posts = []
hashtags = []

BASE_DIR = "instagram_dataset/pl"

# Iterate over whole DataFrame
for i, row in tqdm(arch_df.iterrows()):
    profile_posts = []
    profile_hashtags = []
    
    # Iterate over all categories in base directory
    for cat_dir in os.listdir(BASE_DIR):
        whole_cat_dir = os.path.join(BASE_DIR, cat_dir)
        
        # If profile exists in the database
        if i in os.listdir(whole_cat_dir):
            profile_path = os.path.join(whole_cat_dir, i)
            profile_config_path = os.path.join(whole_cat_dir, i, f"{i}.toml")
            
            # Check if there's a .toml file - if not, omit the profile
            is_present = False            
            if os.path.exists(profile_config_path):
                is_present = True
                for file in os.listdir(profile_path):
                    if not file.endswith(".toml"):
                        with open(os.path.join(profile_path, file), "r") as post_f:
                            read_text = post_f.read()
                            profile_posts.append(remove_stopwords(clean_up_text(read_text)))
                            profile_hashtags.append(extract_hashtags(read_text))
            else:
                available_arch_df = available_arch_df.drop(i, axis=0)
                print(f"Profile {i} has no posts.")
            # Create new list for a given user    
            if is_present:
                # Merge lists - a single list for a single influencer
                profile_hashtags = list(itertools.chain.from_iterable(profile_hashtags))
                hashtags.append(profile_hashtags)
                posts.append(list(itertools.chain.from_iterable([profile_posts])))
            break

9it [00:00, 39.30it/s]

Profile zaradne_warsztaty has no posts.


24it [00:00, 47.26it/s]

Profile ilona_browstylist has no posts.
Profile pracownia.lepiej has no posts.


35it [00:00, 44.27it/s]

...

472it [00:10, 47.83it/s]

Profile zdrowoczylisexy has no posts.
Profile rutynowa has no posts.
Profile aga.lanius has no posts.
Profile balickadesign has no posts.


504it [00:11, 55.86it/s]

Profile nowinki.sklepowe has no posts.
Profile moda_na_klasyki has no posts.


508it [00:11, 45.53it/s]


In [5]:
# Show the current, filtered out database
print(available_arch_df.head())
print(f"Available dataset length: {len(available_arch_df)}")

                     innocent  sage  explorer  outlaw  magician  hero  lover  \
id                                                                             
marek_grodzki            -1.0  -1.0      -1.0    -1.0      -1.0  -1.0   -1.0   
vege_style_life          -1.0  -1.0       1.0    -1.0      -1.0  -1.0   -1.0   
oliwka__2007             -1.0  -1.0      -1.0    -1.0      -1.0  -1.0   -1.0   
z_przestrzeni_serca       1.0  -1.0      -1.0    -1.0      -1.0  -1.0   -1.0   
snatch.machine           -1.0  -0.5       0.5     0.5       0.0   1.0   -1.0   

                     jester  everyman  caregiver  ...  protective  generous  \
id                                                ...                         
marek_grodzki           1.0      -1.0       -1.0  ...         0.0       0.5   
vege_style_life        -1.0      -1.0       -1.0  ...         1.0       1.0   
oliwka__2007           -1.0       1.0       -1.0  ...         0.0       0.0   
z_przestrzeni_serca    -1.0      -1.0       

In [6]:
# Map usernames to indices
users = list(available_arch_df.index.values)
user_indices = {k: users.index(k) for k in users}

In [23]:
# Get the word count and create a dataframe, where columns are archetypes/traits, and rows are single words
# Initialize a word DataFrame
word_df = pd.DataFrame()

# Iterate over all of the traits/archetypes
for trait in tqdm(trait_list):
    # Test procedure for a single trait
    subset_df = available_arch_df[available_arch_df[trait] != 0][trait]
    subset_indices = [user_indices[idx] for idx in subset_df.index.values]

    # Get all posts for the list of influencers
    f = operator.itemgetter(*subset_indices)
    sublist = list(f(posts))
    post_list = []
    for user in sublist:
        for post in user:
            post_list.append(" ".join(post))

    # SPMF - get the most frequent sequences
    spmf = Spmf("PrefixSpan", input_direct=post_list,
                output_filename="output.txt", arguments=[0.00025, 3], input_type="text")
    spmf.run()
    spmf_df = spmf.to_pandas_dataframe(pickle=False)

    # Normalize the data for all lengths
    spmf_df.loc[spmf_df["pattern"].map(len) == 3, "sup"] = spmf_df.loc[spmf_df["pattern"].map(len) == 3].sup / spmf_df[spmf_df["pattern"].map(len) == 3]["sup"].sum()
    spmf_df.loc[spmf_df["pattern"].map(len) == 2, "sup"] = spmf_df.loc[spmf_df["pattern"].map(len) == 2].sup / spmf_df[spmf_df["pattern"].map(len) == 2]["sup"].sum()
    spmf_df.loc[spmf_df["pattern"].map(len) == 1, "sup"] = spmf_df.loc[spmf_df["pattern"].map(len) == 1].sup / spmf_df[spmf_df["pattern"].map(len) == 1]["sup"].sum()

    # Convert lists to tuples
    spmf_df["pattern"] = spmf_df["pattern"].apply(lambda x: str(x))

    sublist = list(f(hashtags))

    # Counter to calculate each word occurrences
    sublist = list(itertools.chain.from_iterable(sublist))
    trait_ctr = Counter(sublist)
    trait_total = sum(trait_ctr.values())
    trait_ctr = {k: float(v / trait_total) for k, v in trait_ctr.items() if v >= 1}
    trait_ctr = {trait: trait_ctr}

    tmp_df = pd.DataFrame.from_dict(trait_ctr, orient="columns")

    # Change column name
    spmf_df = spmf_df.rename(columns={"sup": trait})

    # Change index to pattern
    spmf_df = spmf_df.reset_index(drop=True)
    spmf_df = spmf_df.set_index("pattern")

    # Append the hashtags
    spmf_df = spmf_df.append(tmp_df)
    print(spmf_df)

    spmf_df = spmf_df.transpose()
    spmf_df.to_pickle(f"dfs/{trait}.pickle")

    # Append the dataframe to word_df
    word_df = word_df.append(spmf_df)

  0%|          | 0/37 [00:00<?, ?it/s]

>/media/maciek/HDD_Linux/Praca_magisterska/instagram_analysis/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 6171 ms
 Frequent sequences count : 1862729
 Max memory (mb) : 1029.111328125
 minsup = 4 sequences.
 Pattern count : 1862729

Post-processing to show result in terms of string values.
Post-processing completed.

                                         innocent
['zainspirowany']                    1.298465e-05
['wczorajszym']                      5.193861e-05
['wczorajszym', 'poście']            2.412996e-06
['wczorajszym', 'poście', 'innymi']  4.845665e-07
['wczorajszym', 'naprawdę']          1.930397e-06
...                                           ...
#여성                                  3.679975e-06
#ﬁtnessgirl                          3.679975e-06
#ａｅｓｔｈｅｔｉｃ                           5.151965e-05
#𝐞𝐜𝐨𝐃𝐲𝐰                              3.679975e-06
#𝕠𝕔𝕙𝕛𝕒𝕜𝕗𝕒𝕛𝕟𝕚𝕖𝕓𝕪ć𝕞𝕒𝕞ą               

  3%|▎         | 1/37 [00:22<13:33, 22.59s/it]

>/media/maciek/HDD_Linux/Praca_magisterska/instagram_analysis/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 7880 ms
 Frequent sequences count : 3318340
 Max memory (mb) : 1131.62109375
 minsup = 3 sequences.
 Pattern count : 3318340

Post-processing to show result in terms of string values.
Post-processing completed.

                                             sage
['zainspirowany']                    1.159027e-05
['wczorajszym']                      5.505379e-05
['wczorajszym', 'poście']            2.071720e-06
['wczorajszym', 'poście', 'innymi']  3.600215e-07
['wczorajszym', 'naprawdę']          1.657376e-06
...                                           ...
#інтер                               4.562231e-06
#ねこ                                  9.124462e-06
#猫                                   9.124462e-06
#ﬁtnessgirl                          4.562231e-06
#𝕠𝕔𝕙𝕛𝕒𝕜𝕗𝕒𝕛𝕟𝕚𝕖𝕓𝕪ć𝕞𝕒𝕞ą                

  5%|▌         | 2/37 [01:30<28:48, 49.37s/it]

...

 95%|█████████▍| 35/37 [1:05:54<03:53, 116.76s/it]

>/media/maciek/HDD_Linux/Praca_magisterska/instagram_analysis/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 6317 ms
 Frequent sequences count : 1156266
 Max memory (mb) : 1158.98095703125
 minsup = 5 sequences.
 Pattern count : 1156266

Post-processing to show result in terms of string values.
Post-processing completed.

                            egocentric
['zainspirowany']             0.000013
['wczorajszym']               0.000051
['wczorajszym', 'poście']     0.000002
['wczorajszym', 'przepis']    0.000002
['odnośnie']                  0.000094
...                                ...
#ﬁtnessgirl                   0.000003
#ａｅｓｔｈｅｔｉｃ                    0.000042
#ｔｈｏｕｇｈｔｓ                     0.000003
#𝐞𝐜𝐨𝐃𝐲𝐰                       0.000003
#𝕠𝕔𝕙𝕛𝕒𝕜𝕗𝕒𝕛𝕟𝕚𝕖𝕓𝕪ć𝕞𝕒𝕞ą          0.000003

[1213760 rows x 1 columns]


 97%|█████████▋| 36/37 [1:08:01<01:59, 119.89s/it]

>/media/maciek/HDD_Linux/Praca_magisterska/instagram_analysis/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 6562 ms
 Frequent sequences count : 1164640
 Max memory (mb) : 1242.158203125
 minsup = 5 sequences.
 Pattern count : 1164640

Post-processing to show result in terms of string values.
Post-processing completed.

                            allocentric
['zainspirowany']              0.000013
['wczorajszym']                0.000051
['wczorajszym', 'poście']      0.000002
['wczorajszym', 'przepis']     0.000002
['odnośnie']                   0.000090
...                                 ...
#ﬁtnessgirl                    0.000003
#ａｅｓｔｈｅｔｉｃ                     0.000042
#ｔｈｏｕｇｈｔｓ                      0.000003
#𝐞𝐜𝐨𝐃𝐲𝐰                        0.000003
#𝕠𝕔𝕙𝕛𝕒𝕜𝕗𝕒𝕛𝕟𝕚𝕖𝕓𝕪ć𝕞𝕒𝕞ą           0.000003

[1221534 rows x 1 columns]


100%|██████████| 37/37 [1:10:13<00:00, 113.89s/it]


In [24]:
# Save word_df to file
word_df.to_pickle("word_trait_array.pickle")

In [27]:
# Check the resulting DataFrame
word_df

,['zainspirowany'],['wczorajszym'],"['wczorajszym', 'poście']","['wczorajszym', 'poście', 'innymi']","['wczorajszym', 'naprawdę']","['wczorajszym', 'przepis']","['wczorajszym', 'innymi']",['odnośnie'],"['odnośnie', 'życia']","['odnośnie', 'naprawdę']",...,"['podkład', 'serii']","['konkursu', 'udział', 'nagrody']","['this', 'hope', 'you']","['zadowolony', 'temu']","['przeczytaniu', 'czytając']","['potoczą', 'czekam']","['help', 'and', 'with']","['noszenia', 'maseczek']","['best', 'much']","['care', 'it']"
innocent,0.000013,0.000052,0.000002,4.845665e-07,0.000002,0.000002,0.000002,0.000081,0.000002,0.000003,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
sage,0.000012,0.000055,0.000002,3.600215e-07,0.000002,0.000001,0.000002,0.000113,0.000003,0.000002,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
explorer,0.000016,0.000059,0.000002,3.269538e-07,0.000001,0.000000,0.000002,0.000088,0.000002,0.000003,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
outlaw,0.000014,0.000057,0.000003,3.827676e-07,0.000002,0.000000,0.000002,0.000096,0.000003,0.000003,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
magician,0.000015,0.000059,0.000003,3.768889e-07,0.000002,0.000000,0.000002,0.000100,0.000003,0.000003,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
hero,0.000018,0.000074,0.000003,3.792890e-07,0.000002,0.000000,0.000002,0.000081,0.000003,0.000002,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
lover,0.000012,0.000046,0.000003,5.350438e-07,0.000000,0.000000,0.000002,0.000085,0.000003,0.000003,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
jester,0.000017,0.000058,0.000003,3.576034e-07,0.000002,0.000000,0.000002,0.000092,0.000003,0.000003,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
everyman,0.000010,0.000060,0.000002,4.506976e-07,0.000002,0.000002,0.000002,0.000093,0.000003,0.000003,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
caregiver,0.000012,0.000052,0.000002,3.379707e-07,0.000001,0.000000,0.000002,0.000106,0.000003,0.000003,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000


In [26]:
# Replace all NaN with 0.0
word_df = word_df.fillna(0.0)

In [28]:
# Save non-NaN word_df to file
word_df.to_pickle("word_trait_array_no_nan.pickle")

In [51]:
# Save the results to a pickle
import pickle

with open("influencer_index_map.pickle", "wb") as f:
    pickle.dump(user_indices, f)
    
word_df.to_pickle("word_frequency_table.pickle")

In [40]:
# Method for creating an output vector for dot product calculation
# Word map - to easily create output vectors
word_map = word_df.columns.tolist()

def get_trait_dot_product(post_text: str, word_map: list, word_dataframe: pd.DataFrame) -> list:
    # Filter out the text
    filtered_post = remove_stopwords(clean_up_text(post_text))
    
    filtered_post = [" ".join(pst) for pst in filtered_post]
    filtered_post.extend(extract_hashtags(post_text))
    
    # Create a vector for dot product vector
    post_vector = [0] * len(word_map)
    
    # Calculate word occurrences
    spmf = Spmf("PrefixSpan", input_direct=filtered_post,
                output_filename="output.txt", arguments=[0.00025, 3], input_type="text")
    spmf.run()
    spmf_df = spmf.to_pandas_dataframe(pickle=False)
    
    for idx, row in tqdm(spmf_df.iterrows()):
        phrase = str(row["pattern"])
        freq = int(row["sup"])
        try:
            post_vector[word_map.index(phrase)] = freq
        except:
            continue
    
    # Calculate dot product for a given text
    word_dot = word_dataframe.dot(post_vector)
    return word_dot.tolist()

In [41]:
# Test the trait dot_product
print(get_trait_dot_product("Cześć reasda  asdasda         #hello #man", word_map, word_df))

0it [00:00, ?it/s]

>/media/maciek/HDD_Linux/Praca_magisterska/instagram_analysis/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 2 ms
 Frequent sequences count : 85
 Max memory (mb) : 8.991737365722656
 minsup = 1 sequences.
 Pattern count : 85

Post-processing to show result in terms of string values.
Post-processing completed.



85it [00:07, 12.06it/s]


[0.00016620354740696497, 0.00012749299512918806, 0.0001690490664915492, 0.00014584830353522058, 0.0001594157228390828, 0.00015536049182692842, 0.00018260170915199766, 0.00016443368012853234, 0.00016633483706746642, 0.00010936498436992098, 0.00015111084902183367, 0.00016939103921402557, 0.00017278599281551573, 0.0001591613036986155, 0.00017426378862227474, 0.00016502322112468682, 0.0001605453602390891, 0.0001723152627698697, 0.0001679472645589285, 0.00016507620641309755, 0.00016276248092296895, 0.00015667617021652205, 0.00016640781007321942, 0.00016311498103788345, 0.00015501571392168522, 0.00015649754213106778, 0.00016995963458678562, 0.00016629475070394366, 0.00018173183166301588, 0.00018088433398864235, 0.00015980665583775904, 0.00016073035875016072, 0.00016210352180292368, 0.00014350258416762895, 0.00015313004569130333, 0.00017351924885445163, 0.00015830908779736435]


In [42]:
# Method for calculating the dot product of trait <-> influencer relation
def get_influencer_dot_product(trait_output: list, influencer_dataframe: pd.DataFrame) -> pd.DataFrame:
    return influencer_dataframe.dot(trait_output)

In [43]:
# Method for calculating the similarity
def calculate_similarity(post_text: str, 
                         word_map: list, 
                         word_dataframe: pd.DataFrame,
                         influencer_dataframe: pd.DataFrame) -> pd.DataFrame:
    # Calculate word-trait dot product
    post_result = get_trait_dot_product(post_text, word_map, word_dataframe)
    
    # Calculate trate-influencer dot-product
    return get_influencer_dot_product(post_result, influencer_dataframe)

In [44]:
# Test the method
sim_df = calculate_similarity("""Jak to jest być skrybą, dobrze? 
A, wie pan, moim zdaniem to nie ma tak, że dobrze, albo że niedobrze. 
Gdybym miał powiedzieć, co cenię w życiu najbardziej, powiedziałbym, że ludzi. 
Ludzi, którzy podali mi pomocną dłoń, kiedy sobie nie radziłem, kiedy byłem sam, i co ciekawe, to właśnie przypadkowe spotkania wpływają na nasze życie. 
Chodzi o to, że kiedy wyznaje się pewne wartości, nawet pozornie uniwersalne, bywa, że nie znajduje się zrozumienia, 
które by tak rzec, które pomaga się nam rozwijać. 
Ja miałem szczęście, by tak rzec, ponieważ je znalazłem, i dziękuję życiu! 
Dziękuję mu; życie to śpiew, życie to taniec, życie to miłość! 
Wielu ludzi pyta mnie o to samo: ale jak ty to robisz, skąd czerpiesz tę radość? 
A ja odpowiadam, że to proste! To umiłowanie życia. 
To właśnie ono sprawia, że dzisiaj na przykład buduję maszyny, a jutro – kto wie? 
Dlaczego by nie – oddam się pracy społecznej i będę, ot, choćby, sadzić... doć— m-marchew...""", word_map, word_df, available_arch_df)
print("Maximum similarity:\n"
        f"User: {sim_df.idxmax()}\n"
        f"Similarity score: {sim_df.max()}")

1it [00:00,  4.30it/s]

>/media/maciek/HDD_Linux/Praca_magisterska/instagram_analysis/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 14 ms
 Frequent sequences count : 2445
 Max memory (mb) : 10.53741455078125
 minsup = 1 sequences.
 Pattern count : 2445

Post-processing to show result in terms of string values.
Post-processing completed.



2445it [03:20, 12.21it/s]


Maximum similarity:
User: muzykujemy
Similarity score: 6.420715563113335e-05
